In [1]:
import pandas as pd
import numpy as np
import datetime 
import re 
from icecream import ic 

## Задание путей и имени исследуемого 


In [2]:
dataDir = 'D:\\dataPoly\\'
faceData = dataDir + 'facereader\\'
polyData = dataDir + 'polygraph\\'
timeData = dataDir + 'time\\'
polyChannels = ["ABDOMINAL_RESP", "ABS_BLOOD_VOLUME", "BLOOD_VOLUME", "EDA", "HEART_RATE", "PLE", "THORACIC_RESP", "TONIC_EDA", "TREMOR"]
subject = r'Базаев Кирилл'

## Импорт файла timeline

In [3]:
custom_date_parser = lambda x: datetime.datetime.strptime(x, "%M:%S:%f") #парсер для времени
timeline = pd.read_excel(timeData + 'timeline.xlsx', parse_dates = list(range(2,20)), date_parser = custom_date_parser) #парсим время в колонках 2-19
timeline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      56 non-null     int64         
 1   name    56 non-null     object        
 2   C0_1    56 non-null     datetime64[ns]
 3   C1_1    56 non-null     datetime64[ns]
 4   C2_1    56 non-null     datetime64[ns]
 5   C3_1    56 non-null     datetime64[ns]
 6   C4_1    56 non-null     datetime64[ns]
 7   C5_1    56 non-null     datetime64[ns]
 8   C0_2    56 non-null     datetime64[ns]
 9   C4_2    56 non-null     datetime64[ns]
 10  C5_2    56 non-null     datetime64[ns]
 11  C2_2    56 non-null     datetime64[ns]
 12  C1_2    56 non-null     datetime64[ns]
 13  C3_2    56 non-null     datetime64[ns]
 14  C0_3    56 non-null     datetime64[ns]
 15  C5_3    56 non-null     datetime64[ns]
 16  C3_3    56 non-null     datetime64[ns]
 17  C1_3    56 non-null     datetime64[ns]
 18  C2_3    56 n

In [4]:
timeline.head()

,id,name,C0_1,C1_1,C2_1,C3_1,C4_1,C5_1,C0_2,C4_2,C5_2,C2_2,C1_2,C3_2,C0_3,C5_3,C3_3,C1_3,C2_3,C4_3
0,1,Базаев Кирилл,1900-01-01 00:01:05.302,1900-01-01 00:01:25.656,1900-01-01 00:01:42.966,1900-01-01 00:01:59.607,1900-01-01 00:02:14.862,1900-01-01 00:02:28.700,1900-01-01 00:03:04.222,1900-01-01 00:03:24.951,1900-01-01 00:03:43.267,1900-01-01 00:04:01.486,1900-01-01 00:04:16.666,1900-01-01 00:04:30.894,1900-01-01 00:05:35.598,1900-01-01 00:05:52.870,1900-01-01 00:06:07.758,1900-01-01 00:06:22.134,1900-01-01 00:06:36.667,1900-01-01 00:06:52.270
1,2,Барсукова Мария,1900-01-01 00:00:58.441,1900-01-01 00:01:20.377,1900-01-01 00:01:37.649,1900-01-01 00:01:51.945,1900-01-01 00:02:08.205,1900-01-01 00:02:32.106,1900-01-01 00:03:18.457,1900-01-01 00:03:32.896,1900-01-01 00:03:48.295,1900-01-01 00:04:02.601,1900-01-01 00:04:18.021,1900-01-01 00:04:35.321,1900-01-01 00:04:51.897,1900-01-01 00:05:12.297,1900-01-01 00:05:28.057,1900-01-01 00:05:47.322,1900-01-01 00:06:03.922,1900-01-01 00:06:22.081
2,3,Белоус Полина,1900-01-01 00:03:29.261,1900-01-01 00:03:43.350,1900-01-01 00:04:02.956,1900-01-01 00:04:19.804,1900-01-01 00:04:34.109,1900-01-01 00:04:51.717,1900-01-01 00:05:33.717,1900-01-01 00:05:48.357,1900-01-01 00:06:03.545,1900-01-01 00:06:18.301,1900-01-01 00:06:32.620,1900-01-01 00:06:47.350,1900-01-01 00:07:35.167,1900-01-01 00:07:50.820,1900-01-01 00:08:05.837,1900-01-01 00:08:23.700,1900-01-01 00:08:39.181,1900-01-01 00:08:59.517
3,4,Богеварова Елена,1900-01-01 00:00:44.229,1900-01-01 00:01:04.277,1900-01-01 00:01:23.597,1900-01-01 00:01:40.029,1900-01-01 00:01:57.810,1900-01-01 00:02:11.949,1900-01-01 00:02:45.943,1900-01-01 00:03:03.429,1900-01-01 00:03:17.622,1900-01-01 00:03:32.230,1900-01-01 00:03:46.561,1900-01-01 00:04:00.278,1900-01-01 00:04:41.557,1900-01-01 00:04:55.781,1900-01-01 00:05:09.834,1900-01-01 00:05:26.360,1900-01-01 00:05:40.885,1900-01-01 00:05:55.237
4,5,Браславская Элеонора,1900-01-01 00:02:07.703,1900-01-01 00:02:23.513,1900-01-01 00:02:39.265,1900-01-01 00:02:54.249,1900-01-01 00:03:07.977,1900-01-01 00:03:22.169,1900-01-01 00:04:17.569,1900-01-01 00:04:33.737,1900-01-01 00:04:47.859,1900-01-01 00:05:03.025,1900-01-01 00:05:18.326,1900-01-01 00:05:33.097,1900-01-01 00:06:15.742,1900-01-01 00:06:29.553,1900-01-01 00:06:43.321,1900-01-01 00:06:57.281,1900-01-01 00:07:11.226,1900-01-01 00:07:24.909


## Импорт данных полиграфа

In [47]:
polyDict = {}
readColsP = ['Time', 'Value']

for p in polyChannels:
    pathP1 = polyData + subject + '\\' + 'Фигуры_' + p + '.txt' #путь к файлу
    df = pd.read_table(pathP1 ,sep='\s+', usecols = readColsP)
    df['Value'] = df['Value'].replace(',', '.', regex = True).astype(float)
    df['Time'] = df['Time'].apply(lambda x: datetime.datetime.utcfromtimestamp(x//1000).replace(microsecond=x%1000*1000))
    polyDict[p] = df

for p in polyChannels:
    print(polyDict[p])

                         Time   Value
0     1970-01-01 00:00:00.066  8129.0
1     1970-01-01 00:00:00.100  8282.0
2     1970-01-01 00:00:00.132  8339.0
3     1970-01-01 00:00:00.164  8399.0
4     1970-01-01 00:00:00.195  8240.0
...                       ...     ...
13340 1970-01-01 00:07:07.696  8564.0
13341 1970-01-01 00:07:07.727  8514.0
13342 1970-01-01 00:07:07.759  8464.0
13343 1970-01-01 00:07:07.791  8417.0
13344 1970-01-01 00:07:07.823  8374.0

[13345 rows x 2 columns]
                         Time  Value
0     1970-01-01 00:00:00.066    0.0
1     1970-01-01 00:00:00.100    0.0
2     1970-01-01 00:00:00.132    0.0
3     1970-01-01 00:00:00.164    0.0
4     1970-01-01 00:00:00.195    0.0
...                       ...    ...
13340 1970-01-01 00:07:07.696   49.0
13341 1970-01-01 00:07:07.727   49.0
13342 1970-01-01 00:07:07.759   49.0
13343 1970-01-01 00:07:07.791   48.0
13344 1970-01-01 00:07:07.823   48.0

[13345 rows x 2 columns]
                         Time   Value
0     1970

In [54]:
#bd_vl['Time'].dt.time

In [8]:
#emotions['Video Time'] = pd.to_datetime(emotions['Video Time'])
#pd.to_datetime(emotions['Video Time']).dt.time

## Импорт FaceReader

In [37]:
#cols = ['Neutral', 'Happy', 'Sad', 'Angry', 'Surprised', 'Scared',
#       'Disgusted', 'Contempt', 'Valence', 'Arousal', 'Y - Head Orientation', 
#        'X - Head Orientation','Z - Head Orientation', 'Quality']

readColsF = ['Video Time', 'Neutral', 'Happy', 'Sad', 'Angry', 'Surprised', 'Scared', 'Disgusted', 'Contempt', 'Valence', 'Arousal'] 
colsF = ['Neutral', 'Happy', 'Sad', 'Angry', 'Surprised', 'Scared', 'Disgusted', 'Contempt', 'Valence', 'Arousal'] 
#conv = dict.fromkeys(cols, lambda x: pd.to_numeric(x, errors='coerce'))
na = ["FIND_FAILED", "FIT_FAILED"] #значения NA
pathF1 = faceData + subject + '.txt'

emotions = pd.read_table(pathF1, sep='\t', skiprows=8, index_col=False, parse_dates = ['Video Time'], na_values = na, usecols = readColsF)
#converters= conv

emotions[colsF] = emotions[colsF].apply(pd.to_numeric,errors='coerce').fillna(emotions)

#for i in range(26, 68):
#    emotions.iloc[:,i] = pd.to_numeric(emotions.iloc[:,i], errors='coerce').fillna(0, downcast='infer')


In [38]:
emotions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Video Time  10681 non-null  datetime64[ns]
 1   Neutral     10651 non-null  float64       
 2   Happy       10651 non-null  float64       
 3   Sad         10651 non-null  float64       
 4   Angry       10651 non-null  float64       
 5   Surprised   10651 non-null  float64       
 6   Scared      10651 non-null  float64       
 7   Disgusted   10651 non-null  float64       
 8   Contempt    10651 non-null  float64       
 9   Valence     10651 non-null  float64       
 10  Arousal     10651 non-null  float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 918.0 KB


In [39]:
emotions.iloc[:10] #проверка значений NaN

,Video Time,Neutral,Happy,Sad,Angry,Surprised,Scared,Disgusted,Contempt,Valence,Arousal
0,2023-04-24 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-04-24 00:00:00.040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-24 00:00:00.080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-24 00:00:00.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-04-24 00:00:00.160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-04-24 00:00:00.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-04-24 00:00:00.240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2023-04-24 00:00:00.280,0.506445,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.606562
8,2023-04-24 00:00:00.320,0.506471,1.000000e-09,0.000006,0.000000,1.940000e-07,4.000000e-09,0.000010,0.000025,-0.000010,0.606562
9,2023-04-24 00:00:00.360,0.504043,1.900000e-08,0.000012,0.004911,1.720000e-07,1.230000e-07,0.000284,0.000022,-0.004911,0.606548


## Формирование итогового выходного массива

#### Скелет итогового датафрейма

In [40]:
subID = timeline[timeline['name'] == subject]['id'][0]
test = pd.DataFrame(columns= ['id', 'name', 'time', 'stimul', 'trial'] + list(emotions.columns)[1:] + polyChannels)
test

,id,name,time,stimul,trial,Neutral,Happy,Sad,Angry,Surprised,...,Arousal,ABDOMINAL_RESP,ABS_BLOOD_VOLUME,BLOOD_VOLUME,EDA,HEART_RATE,PLE,THORACIC_RESP,TONIC_EDA,TREMOR


#### Функция для конвертации объекта datetime.time в миллисекунды

In [41]:
def totalMiliSec(time):
    return (time.microsecond + time.second * 1000000 + time.minute * 60 * 1000000 + time.hour * 60 * 60 * 1000000)/1000


#### Функция для фильтрации нулевых стимулов

In [42]:
def filter_stim(s): 
    return not re.search(r'C0_\d', s)

In [43]:
stimulRead = filter(filter_stim, list(timeline.columns[2:])) #фильтруем список стимулов из датафрейма, чтоб убрать нулевые

for s in stimulRead:
    print(totalMiliSec(timeline.loc[timeline['id'] == subID][s][0].time())) #время предявления каждого (ненулевого) стимула в мс

85656.0
102966.0
119607.0
134862.0
148700.0
204951.0
223267.0
241486.0
256666.0
270894.0
352870.0
367758.0
382134.0
396667.0
412270.0


#### Функция для усреднения строк файлов полиграфа

In [53]:
def sortPoly(df, t):
    df1 = df.copy(deep=True)
    df1['Time'] = df1['Time'].map(lambda x: (x.microsecond + x.second * 1000000 + x.minute * 60 * 1000000 +
                                         x.hour * 60 * 60 * 1000000) / 1000)
    rows = df1[(t <= df1['Time']) & (df1['Time'] < t + 50)]
    #print(rows['Value'])
    if len(rows) > 1:
        tot = rows['Value'].mean()
    elif len(rows) != 0:
        tot = rows.iloc[0]['Value']
    else:
        tot = 0
    return tot

#### Формирование итогового датафрейма

In [51]:
#конечное время
end_time = 0 #в миллисекундах, самая ппоследняя временная метка из двух файлов
if (polyDict['EDA']['Time'].iloc[-1].time() > emotions['Video Time'].iloc[-1].time()):
    end_time = totalMiliSec(polyDict['EDA']['Time'].iloc[-1].time())
else:
    end_time = totalMiliSec(emotions['Video Time'].iloc[-1].time())

print (end_time)

df2 = emotions.copy(deep=True)
df2['Video Time'] = df2['Video Time'].map(
    lambda x: (x.microsecond + x.second * 1000000 + x.minute * 60 * 1000000 +
               x.hour * 60 * 60 * 1000000) / 1000)

427823.0


In [54]:
# очистка итогового датафрейма
ic()  # start
test.drop(test.index, inplace=True)

time = 0  # начальное время
pTotal = {}
while time < int(end_time):
 
    for k in polyDict:
        tot = sortPoly(polyDict[k], time)
        pTotal[k] = tot
    # ic(pTotal)

    eRows = df2[(time <= df2['Video Time']) & (df2['Video Time'] < time + 50)]

    if len(eRows) > 1:
        # ic('if1')
        # ic()
        # ic(eRows)
        # eTotal = pd.DataFrame(np.nanmean(eRows))
        eTotal = eRows.mean().to_frame().T
        # ic(eTotal)
    elif len(eRows) != 0:
        # ic('if2')
        # ic()
        # ic(eRows)
        eTotal = eRows.reset_index(drop=True)
        # ic(eTotal)
    else:
        # ic('if3')
        # ic()
        eTotal = pd.DataFrame(0, index=[0],
                              columns=['Neutral', 'Happy', 'Sad', 'Angry', 'Surprised', 'Scared', 'Disgusted',
                                       'Contempt', 'Valence', 'Arousal'])

    time += 50  # временной шаг (может регулироваться)
    test.loc[len(test.index)] = [subID, subject, time, 0, 0, eTotal['Neutral'][0], eTotal['Happy'][0],
                                 eTotal['Sad'][0], eTotal['Angry'][0], eTotal['Surprised'][0],
                                 eTotal['Scared'][0], eTotal['Disgusted'][0], eTotal['Contempt'][0],
                                 eTotal['Valence'][0], eTotal['Arousal'][0], pTotal['ABDOMINAL_RESP'], pTotal['ABS_BLOOD_VOLUME'], pTotal['BLOOD_VOLUME'], pTotal['EDA'], pTotal['HEART_RATE'], pTotal['PLE'], pTotal['THORACIC_RESP'], pTotal['TONIC_EDA'], pTotal['TREMOR']]

    # pRows.drop(pRows.index, inplace=True)  # очищаем промежуточные датафреймы
    eRows.drop(eRows.index, inplace=True)
    pTotal.clear()

ic()  # stop
test.to_csv(dataDir + 'file.csv', encoding='utf-8')

ic| 2139862946.py:2 in <module> at 15:18:37.764
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, 

C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, 

C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, 

C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, inplace=True)
C:\Users\tassy\AppData\Local\Temp\ipykernel_7864\2139862946.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eRows.drop(eRows.index, 

KeyboardInterrupt: 